# Nuclear Open Data

Pulling data from safecast.org and doing stuff with it

Imports go here

In [34]:
import matplotlib
import json
import requests
import gmplot
import webbrowser
import numpy

API Call function

In [40]:
def api_call(captured_after = "2017-05-01+00%3A00", captured_before = "2017-05-02+00%3A00"):
    base_url = "https://api.safecast.org/measurements.json"
    api_key = "hj5Nam6xXLATGcvUpP14"
    call_url = base_url + "?captured_after=" + captured_after + "&captured_before=" + captured_before
    # call_url = base_url + "?api_key=" + api_key
    print call_url
    r = requests.get(call_url)
    return r

In [41]:
r =  api_call()
the_json = json.loads(r.text)

https://api.safecast.org/measurements.json?captured_after=2017-05-01+00%3A00&captured_before=2017-05-02+00%3A00


test looking into the json output

In [42]:
lats = []
longs = []
set_size = len(the_json)
count = 0
sum = 0
for i in range(0, set_size):
    if the_json[i]["unit"] == "cpm":
        sum = the_json[i]["value"] + sum
        lats.append(the_json[i]["latitude"])
        longs.append(the_json[i]["longitude"])
        count += 1
average = round(sum / count,2)
print "Number of sites = " + str(count)
print "Average radiation level = " + str(average)

Number of sites = 19
Average radiation level = 36.63


plot the data points into google maps

In [49]:
map_lat = numpy.mean(lats)
map_long = numpy.mean(longs)
print map_lat
print map_long
gmap = gmplot.GoogleMapPlotter(map_lat, map_long, 4)

gmap.scatter(lats, longs, 'cornflowerblue', size = 40, marker=True)

gmap.draw("mymap.html")

36.8260767895
91.2512566316


<iframe src="file:///home/matthew/code1161base/open_data_project/mymap.html">
</iframe>

simple http server turn folder into localhost - python -m SimpleHTTPServer 8000

In [50]:
url = "mymap.html"
webbrowser.open(url)

True